# Ultimate Guide to Merging Data in Pandas
## From semi/anti joins to validating data merges

<img src='images/cover.jpg'></img>

### Introduction

> _The goal of this article is that you come away with a strong knowledge of combining data in pandas using precise methods suited for any question you want to ask about your data._

With each data science project or dataset, you want to perform several analyses and create plots to find insights. Often, the raw data never comes in one massive table but in many separate ones. To answer your questions, you should have the skills to join multiple tables into one and then perform operations on them.

You can acquire these skills by learning different kinds of merge operations such as inner join, left and right joins, self and anti joins, merging on indexes, etc.


### Overview

   I. Introduction
   
  II. [Pandas merge()](#pandas_merge)
     1. [Pandas merge() inner join (question 1)](#question_1)
     2. [Pandas merge() left/right join (question 2)](#question_2)
     3. [Pandas merge() suffixes](#suffixes)
     4. [Pandas merge(), left_on and right_on](#left_right)
 III. [Pandas Merging on Indexes](#merge_index)
 
  IV. [Pandas Joining a Table to Itself. Self Join.](#self_join)
  
   V. [Filtering Joins](#filter_joins)
      1. [Pandas semi-join](#semi_join)
      2. [Pandas anti-join](#anti_join)
      
  VI. [Pandas concat()](#pandas_concat)
      1. [Pandas vertical concatenation](#vertical)
      
 VII. [Pandas append()](#pandas_append)
 
VIII. [Types of Data Mappings](#data_mappings)

  IX. [Verifying Concatenations](#verify)
  
   X. [Conclusion](#conclusion)

> Unfortunately, Medium does have a functionality to hyperlink to parts of the article, so get scrolling!

### Pandas merge() <small id='pandas_merge'></small>
Pandas provide several methods for performing merges on dataframes. Among all the others `merge()` method is the most flexible. It is a dataframe method and the general syntax is as follows: 

`df1.merge(df2, on='common_column')`

When combining tables, there are two terminologies you should be familiar with: The name of the table you use first is called __the left table__ while the other is called __the right table__. In the code snippet above, the left table is `df1` and the right table is `df2`. Also, the verbs join, combine and merge are all used interchangeably.

Now let's see how we perform an inner join:

An inner join will only return rows that have matching values in both tables. During the joining process, you will have to know the common table name which exists in both tables.
<img src='images/inner_join.png'></img>

In [1]:
# Load necessary libraries
import pandas as pd
import numpy as np

In [2]:
# Enable multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Load necessary dataframes
user_usage = pd.read_csv('data/user_usage.csv')
user_devices = pd.read_csv('data/user_device.csv').drop('user_id', axis='columns')

### Basic Exploration

In [4]:
user_usage.info()
user_devices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   outgoing_mins_per_month  240 non-null    float64
 1   outgoing_sms_per_month   240 non-null    float64
 2   monthly_mb               240 non-null    float64
 3   use_id                   240 non-null    int64  
dtypes: float64(3), int64(1)
memory usage: 7.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   use_id            272 non-null    int64  
 1   platform          272 non-null    object 
 2   platform_version  272 non-null    float64
 3   device            272 non-null    object 
 4   use_type_id       272 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 10.8+ KB


In [5]:
user_usage.describe()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
count,240.000000,240.000000,240.000000,240.000000
mean,274.559167,98.968292,3628.602042,23285.516667
std,293.745744,111.172685,4486.311513,624.139253
min,0.500000,0.250000,0.000000,22787.000000
25%,74.590000,29.030000,1132.230000,22888.750000
50%,189.705000,70.775000,1797.975000,22987.500000
75%,336.045000,125.627500,4246.617500,23482.500000
max,1816.630000,906.920000,31146.670000,25220.000000


Let's say we have these two tables:

In [6]:
user_usage.head()
user_devices.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


,use_id,platform,platform_version,device,use_type_id
0,22782,ios,10.2,"iPhone7,2",2
1,22783,android,6.0,Nexus 5,3
2,22784,android,5.1,SM-G903F,1
3,22785,ios,10.2,"iPhone7,2",3
4,22786,android,6.0,ONE E1003,1


This data was downloaded from the [KillBiller](http://www.killbiller.com/) application. [KillBiller](http://www.killbiller.com/) was a free service that compared every mobile tariff in UK and Ireland. The first, `user_usage` table contains monthly statistics of mobile usage of users. `user_devices` table provides details about each users phone such as operating system and phone model.

#### Question 1: <small id='question_1'></small>
How many users use Android OS and how many use iOS?

To answer this question, we will need the information from both tables. There is one linking attribute between both tables: `use_id`. We will use this column in our merge:

In [7]:
usage_w_os = user_usage.merge(user_devices[['platform', 'use_id']], on='use_id', how='inner')
usage_w_os.sample(5)
print(f'Number of users for each OS: {usage_w_os["platform"].value_counts()}')

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform
43,67.15,39.03,3114.67,22865,android
144,60.08,261.33,12458.67,23027,android
17,797.06,7.67,15573.33,22816,android
6,71.59,79.26,519.12,22794,android
118,146.45,48.67,1557.33,22986,android


Number of users for each OS: android    157
ios          2
Name: platform, dtype: int64


It looks like there is a huge difference between the two operating systems in our dataset. 

In the merge above, we used an example of an inner join. In `merge()` function `how` argument is set to `inner` by default so we did not have to write it out. When merging two tables using the `merge()` function, we use `on` argument to specify the common column. If there are multiple, it is also possible to pass a list of columns to the argument and `pandas` will take care of the rest. 

Note that as a right table, I subset the `user_devices` table to exclude irrelevant columns to the question. 

Now, as we explore further, we will notice that number of given users in two datasets is different:

In [8]:
print(f'Dimensions of user_usage table: {user_usage.shape}')
print(f'Dimensions of user_devices table: {user_devices.shape}')
print(f'Dimensions of the joined table: {usage_w_os.shape}')

Dimensions of user_usage table: (240, 4)
Dimensions of user_devices table: (272, 5)
Dimensions of the joined table: (159, 5)


Clearly, matching user IDs in both tables were 159. This means there are user IDs which are in `user_devices` table and not in `user_usage` table and vice versa. So, the next question we want to ask is:

#### Question 2 <small id='question_2'></small>
How many users use Android OS and iOS, including all the users that aren't in `user_usage` table?

We can answer this question by using either a __left__ or a __right__ join. First let's look at the general case of one-sided joins:
<img src='images/left_right_join.png'></img>

A __left join__ will return all of the rows of the left table including matching rows from the right table.
A __right join__ will return all of the rows of the right table including matching rows from the left table. Note that in both types of joins, `pandas` will put an `NaN` value to the cells with no match.

In [9]:
usage_w_os2 = user_usage.merge(user_devices[['platform', 'use_id']], on='use_id', how='right')
usage_w_os2.sample(5)
print(f'Number of users for each OS: {usage_w_os2["platform"].value_counts()}')

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform
33,420.21,274.76,519.12,22852,android
92,250.87,149.37,519.12,22945,android
173,NaN,NaN,NaN,22810,ios
193,NaN,NaN,NaN,22847,ios
28,135.09,42.02,5191.12,22833,android


Number of users for each OS: android    184
ios         88
Name: platform, dtype: int64


The same result can be achieved using a left join. We can just swap the tables and pass `left` keyword for the `how` parameter:

In [10]:
usage_w_os2 = user_devices[['platform', 'use_id']].merge(user_usage, on='use_id', how='left')
usage_w_os2.sample(5)
print(f'Number of users for each OS: {usage_w_os2["platform"].value_counts()}')

,platform,use_id,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb
236,android,23018,101.59,84.41,5191.12
170,ios,22952,NaN,NaN,NaN
234,android,23016,99.81,403.78,3114.67
154,android,22936,165.90,81.62,3114.67
39,android,22821,NaN,NaN,NaN


Number of users for each OS: android    184
ios         88
Name: platform, dtype: int64


We use an outer join to return the intersection and all the other rows from both tables:
<img src='images/outer_join.png'></img>

#### Pandas merge(), suffixes <small id='suffixes'></small>

Let's say we have columns with identical names in both of the tables. Since in our case, we don't have columns with the same name I will just copy out one column into the other:

In [11]:
user_devices['monthly_mb'] = user_usage['monthly_mb']
user_devices.columns
user_usage.columns

Index(['use_id', 'platform', 'platform_version', 'device', 'use_type_id',
       'monthly_mb'],
      dtype='object')

Index(['outgoing_mins_per_month', 'outgoing_sms_per_month', 'monthly_mb',
       'use_id'],
      dtype='object')

Now we have `monthly_mb` column in both tables. Let's see what happens when we join them:

In [12]:
result = user_devices.merge(user_usage, on='use_id')
result.sample(3)

,use_id,platform,platform_version,device,use_type_id,monthly_mb_x,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb_y
100,22964,android,6.0,SM-G900F,1,1441.29,28.90,29.42,3114.67
124,22994,android,6.0,SM-A300FU,1,1441.29,95.22,207.59,1557.33
81,22933,android,6.0,SM-G900F,1,2076.45,237.19,25.35,3114.67


If we look at the column names, the two columns with the same names have been renamed by adding `_x` and `_y` suffixes. Here, `_x` is for the left table and `_y` is for the right. This is `pandas` default behavior when we don't specify the suffixes. We can control the suffixes easily:

In [13]:
result = user_devices.merge(user_usage, on='use_id', suffixes=('_device', '_stat'))
result.sample(3)

,use_id,platform,platform_version,device,use_type_id,monthly_mb_device,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb_stat
2,22789,android,6.0,SM-G930F,1,519.12,1710.08,136.88,7267.55
106,22970,android,5.0,SM-G900F,1,6015.75,192.58,60.83,3114.67
111,22977,android,5.0,GT-I9505,1,5131.29,360.86,103.92,3114.67


It is much better now. Use such suffixes to indicate the columns table of origin when they have identical names. Note that `suffixes` parameter takes a __tuple__ of values, not a list and order does matter!

In [14]:
# Undo all recent changes
user_usage = pd.read_csv('data/user_usage.csv')
user_devices = pd.read_csv('data/user_device.csv').drop('user_id', axis='columns')

#### Pandas merge(), left_on and right_on <small id='left_right'></small>

There is also a case where you want to join on a common column but the columns have different names. `pandas` provides a handy argument for doing just that without changing the names of the columns. Let's rename the `use_id` columns so that I can show you an example.

In [15]:
user_usage.rename(columns={'use_id': 'user'}, inplace=True)
user_usage.columns
user_devices.columns

Index(['outgoing_mins_per_month', 'outgoing_sms_per_month', 'monthly_mb',
       'user'],
      dtype='object')

Index(['use_id', 'platform', 'platform_version', 'device', 'use_type_id'], dtype='object')

Now, let's combine the tables with an `inner` join:

In [16]:
user_usage.merge(user_devices, left_on='user', right_on='use_id').head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,user,use_id,platform,platform_version,device,use_type_id
0,21.97,4.82,1557.33,22787,22787,android,4.3,GT-I9505,1
1,1710.08,136.88,7267.55,22788,22788,android,6.0,SM-G930F,1
2,1710.08,136.88,7267.55,22789,22789,android,6.0,SM-G930F,1
3,94.46,35.17,519.12,22790,22790,android,5.1,D2303,1
4,71.59,79.26,1557.33,22792,22792,android,5.1,SM-G361F,1


We use `left_on` and `right_on` parameters of the merge function to specify the names of common columns. You should pass the name of the column from the left table to `left_on` and vice versa. Note that when we use differently named common column, `pandas` will keep both of the columns as duplicates, so don't forget to drop one of them:

In [17]:
user_usage.merge(user_devices, left_on='user', right_on='use_id').drop('user', axis='columns').head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,platform_version,device,use_type_id
0,21.97,4.82,1557.33,22787,android,4.3,GT-I9505,1
1,1710.08,136.88,7267.55,22788,android,6.0,SM-G930F,1
2,1710.08,136.88,7267.55,22789,android,6.0,SM-G930F,1
3,94.46,35.17,519.12,22790,android,5.1,D2303,1
4,71.59,79.26,1557.33,22792,android,5.1,SM-G361F,1


In [18]:
# Undo all recent changes
user_usage = pd.read_csv('data/user_usage.csv')
user_devices = pd.read_csv('data/user_device.csv').drop('user_id', axis='columns')

### Pandas merging on indexes <small id='merge_index'></small>

It is also possible to merge on a common index. The syntax for any type of join using indexes is the same. `pandas` is smart enough to know that you are referring to the index:

In [19]:
# Set use_id as indexes
user_usage.set_index('use_id', inplace=True)
user_devices.set_index('use_id', inplace=True)

# Merge on index
user_usage.merge(user_devices, on='use_id', how='outer').head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,platform,platform_version,device,use_type_id
use_id,,,,,,,
22787,21.97,4.82,1557.33,android,4.3,GT-I9505,1.0
22788,1710.08,136.88,7267.55,android,6.0,SM-G930F,1.0
22789,1710.08,136.88,7267.55,android,6.0,SM-G930F,1.0
22790,94.46,35.17,519.12,android,5.1,D2303,1.0
22792,71.59,79.26,1557.33,android,5.1,SM-G361F,1.0


Things will get a little tricky when we have indexes with different names. Similar to columns, we will have to use `left_on` and `right_on` parameters. Also, there are two more parameters to indicate that we are merging on an index:

In [20]:
# Rename the names of the index
user_usage.index.names = ['index_1']
user_devices.index.names = ['index_2']    # Yes, that's how you do it)))

# Merge on index
user_usage.merge(user_devices, left_on='index_1', left_index=True, right_on='index_2', right_index=True).sample(3)

,index_1,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,platform,platform_version,device,use_type_id
23032,23032,227.13,76.94,1038.21,android,5.1,SM-J320FN,1
22891,22891,205.11,273.75,16611.55,android,5.0,SM-N9005,1
22824,22824,208.26,91.76,5191.12,android,6.0,SM-G900F,1


We set `left_index` and `right_index` to `True` to indicate we are merging on indexes. This also gives duplicate values: one as an index and one as a column. So, don't forget to drop the column when you are merging!

In [21]:
# Undo all recent changes
user_usage = pd.read_csv('data/user_usage.csv')
user_devices = pd.read_csv('data/user_device.csv').drop('user_id', axis='columns')

If the tables you are merging have MultiLevel indexes, you can pass a list of index names to the `on` or `left/right_on` parameters and you will be set!

### Pandas joining a table to itself <small id='self_join'></small>

There might be cases where you may want to join a table to itself. Joining a table to itself can be useful when you have:
    1. Hierarchical relationships
    2. Sequential relationships
    3. Graph data

General case of joining a table to itself:
<img src='images/self_join.png'></img>

In [ ]:
# Actual code
sequel.merge(sequel, left_on='id', right_on='sequel_id')

As you can see, a self merge can be pretty useful at times. When you use a self join, you often join on different columns of the same table.

### Filtering joins <small id='filter_joins'></small>

All the joins we had done so far have returned data from both tables. But sometimes, you would want to filter your table based on values on some other table. `pandas` does not provide this functionality directly. But we can engineer the steps pretty easily.
#### Semi-join Pandas <small id='semi_join'></small>

Semi-joins are useful when you want to subset your data based on observations in other tables. Semi-joins:
    1. Returns the intersection of two tables, similar to an inner join.
    2. Returns only the columns from the __left__ table, not the __right__.
    3. No duplicates
Back to our little tables:
<img src='images/semi_join.png'></img>

As you can see, semi-join is exactly like an inner join but it only includes the results from the left table. Now back to our example data, we want to know the stats for all the users which exist in both tables. But we don't want the final table to contain columns of the `user_devices` table. We can approach this problem in three steps:

In [22]:
# First, perform an inner join to find common use_ids
semi_table = user_usage.merge(user_devices, on='use_id')
semi_table.head(3)

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,platform_version,device,use_type_id
0,21.97,4.82,1557.33,22787,android,4.3,GT-I9505,1
1,1710.08,136.88,7267.55,22788,android,6.0,SM-G930F,1
2,1710.08,136.88,7267.55,22789,android,6.0,SM-G930F,1


After we have the inner join, we want to subset our original left table with the values in the final table:

In [23]:
# Second, create a boolean series
in_both = user_usage['use_id'].isin(semi_table['use_id'])
in_both.sample(5)

177    False
174    False
12      True
39      True
233    False
Name: use_id, dtype: bool

The variable `in_both` now is a `Boolean Series` which we can use to subset the left table

In [24]:
# Finally, subset the left table
users_in_both = user_usage[in_both]
users_in_both.sample(3)
users_in_both.shape

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
123,244.93,16.22,20764.45,22992
12,283.30,107.47,15573.33,22806
153,198.59,90.49,5191.12,23041


(159, 4)

The results are as we expected. If you compare this join to the very first `inner` join we did, it is the exact same table without the columns of the right table:

In [25]:
inner = user_usage.merge(user_devices, on='use_id')
inner.sample(3)
inner.shape

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,platform_version,device,use_type_id
120,114.59,69.20,1557.33,22988,android,5.0,SM-G360F,1
51,83.46,114.06,3114.67,22880,android,5.0,GT-I9505,1
61,28.85,30.22,3114.67,22890,android,6.0,SM-G900F,1


(159, 8)

This is a tricky case. Please, reread and practice this section more to really understand it)))

#### Anti-join Pandas <small id='anti_join'></small>

Anti-join is the complete opposite of a semi-join. Anti-join:
    1. Returns the left table, __excluding__ the intersection
    2. Returns only columns from the left table.
<img src='images/anti_join.png'></img>

As an example, let's do the complete opposite of the previous step in semi-join. Let's find out all the users which are only in the `user_usage` table. Step 1:

In [26]:
# Perform a left join
left_joined = user_usage.merge(user_devices, how='left', on='use_id', indicator=True)
left_joined.sample(5)

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,platform_version,device,use_type_id,_merge
120,114.59,69.20,1557.33,22988,android,5.0,SM-G360F,1.0,both
213,583.73,216.52,1441.29,24333,NaN,NaN,NaN,NaN,left_only
158,42.75,46.83,5191.12,23053,android,5.1,Vodafone Smart ultra 6,1.0,both
230,583.73,216.52,1441.29,24764,NaN,NaN,NaN,NaN,left_only
154,198.59,90.49,5191.12,23043,android,6.0,SM-G900F,1.0,both


Now, I introduce you to another helpful parameter: `indicator`. When set to `True`, the resulting table of the merge will have one extra column, `_merge`. It indicates whether a row is a result of both tables or one side. As you see, there is `left_only` string in the rows that did not have a match. Now, we want to subset for those rows:

In [27]:
left_only = left_joined.loc[left_joined['_merge'] == 'left_only', 'use_id']
left_only.sample(3)

228    24728
187    23823
159    23057
Name: use_id, dtype: int64

I used `.loc` subsetting to immediately get the user IDs. Now we can use this pandas `series` to subset the original left table:

In [28]:
left_only_users = user_usage[user_usage['use_id'].isin(left_only)]
left_only_users.sample(5)
left_only_users.shape

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
183,190.08,30.92,369.84,23591
220,205.40,42.08,1045.39,24451
182,583.73,216.52,1441.29,23548
235,260.66,68.44,896.96,25008
169,73.49,6.94,5191.12,23183


(81, 4)

Again, this is a bit more complicated, so get practicing...

### Pandas concat() <small id='pandas_concat'></small>

Finally, we covered most of the `merge` function. Now, `pd.concat()` is used for concatenating/adding two or more tables along a particular axis. It also provides some functionalities to manipulate axes. A common way the function is used is when adding tables vertically.

#### Pandas vertical concatenation of dataframes<small id='vertical'></small>

To add two or more tables vertically, we use `concat()` method, with the `axis` argument set to `index`.
<img src='images/vertical_concat.png'></img>

Let's load one of the sample tables in three chunks:

In [29]:
# Load the sample table in three chunks
user_usage_1, user_usage_2, user_usage_3 = [chunk for chunk in pd.read_csv('data/user_usage.csv', chunksize=80)]

# Concatenate the three chunks vertically
user_usage = pd.concat([user_usage_1, user_usage_2, user_usage_3], axis='index')
user_usage.shape

(240, 4)

Pay attention to how I used the `chunksize` parameter! It is useful to know as it will be immensely helpful when loading large datasets!

When using `concat()` function, we always pass the tables to be concatenated in a list. To specify the concatenation axis, we use `axis` parameter. I highly recommend you pass string values to `axis` parameter such as `index` and `columns`. Passing 0 or 1 can be confusing for a lot of people. So, it is a good practice to improve code readability.

`concat()` function does not pay attention to axis labels. If you have numbered indexes in all of your tables, they will be mixed when you concatenate. So, a helpful paramter for resolving the issue is `ignore_index`:

In [30]:
pd.concat([user_usage_3, user_usage_1, user_usage_2], axis='index', ignore_index=True).head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,60.41,0.80,3114.67,23058
1,60.41,0.80,3114.67,23059
2,46.40,65.90,1557.33,23073
3,38.56,47.65,4025.86,23083
4,747.97,65.84,4867.46,23084


This will reset the index and it will count from 0 to n-1.

Another use case for `concat()` is when you have similar tables but with different column names. When you concatenate tables with differing column names, use `join` parameter with a suitable string `value`.
<img src='images/inner_vertical.png'></img>

In [31]:
# Concat vertically with an inner join
pd.concat([user_usage, user_devices], join='inner', axis='index').head(3)

,use_id
0,22787
1,22788
2,22789


Using `concat` with `inner` value resulted in a one-column dataframe since it is the only common column between tables. Using `left`, `right`, or `outer` would have expected results but values stacked vertically.

### Pandas append()<small id='pandas_append'></small>

`.append()` method:
    1. Simplified version of `concat()` method
    2. Supports: `ignore_index`
    3. Does NOT support: `join` and other differences.

Another difference is that it is a DataFrame method. Meaning that it is called on a DataFrame rather than explicitly like `pd.concat`. Simple example:

In [32]:
user_usage = user_usage_1.append([user_usage_2, user_usage_3], ignore_index=True)
user_usage.shape

(240, 4)

### Types of Data Mappings <small id='data_mappings'></small>

We saw several types of joins above. In each one, we specifically told the methods the type of join we want and just took a leap of faith that it will return the correct result. However, there are methods to further ensure that we get the correct results. To do the task in code, you have to understand 4 types of data mappings in merges:
    1. One to One
    2. One to Many
    3. Many to One
    4. Many to Many
Let's elaborate. When we use a one_to_one mapping, we expect that our join should return one and only one match for each row in the left table. If we are joining a table with 200 rows with a one_to_one mapping, the result should also contain 200 rows.

Similarly, if it is one_to_many, we allow multiple matches for a single row in our left table. Let's how we implement the idea in code:

In [33]:
user_usage.merge(user_devices, on='use_id', validate='one_to_one').sample(3)

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,platform_version,device,use_type_id
140,63.56,26.87,9344.00,23021,android,6.0,SM-G930F,1
32,103.97,234.72,1557.33,22850,android,6.0,SM-A310F,1
51,83.46,114.06,3114.67,22880,android,5.0,GT-I9505,1


`merge()` method provides a helpful parameter for validating these kinds of merges. It accepts only four values:
    1. `one_to_one`
    2. `one_to_many`
    3. `many_to_one`
    4. `many_to_many`

What's more, `pandas` will raise a `MergeError` if we validate on an incorrect mapping:

In [34]:
# Load new sample data
films = pd.read_csv('data/films.csv')
directors = pd.read_csv('data/directors.csv')

In [35]:
films.merge(directors, on='director_id', validate='one_to_one')

MergeError: Merge keys are not unique in either left or right dataset; not a one-to-one merge

As you can see, the merge raised an error. Let's set it to the correct mapping:

In [36]:
films.merge(directors, on='director_id', validate='many_to_many').sample(3)

,id,film,release_date,sequel_id,director_id,name
13,108,Captain America 2,2014-04-04,112.0,208,Anthony and Joe Russo
5,103,Thor,2011-05-06,107.0,203,Kenneth Branagh
4,101,The Incredible Hulk,2008-07-13,NaN,202,Louis Leterrier


#### Verifying concatenations <small id='verify'></small>

It is also possible to validate on axes when using `.concat()` method. It has `verify_integrity` parameter which is set to `False` by default. When set to True, it will raise a ValueError if the axis or columns have duplicate names:

In [37]:
# Set a new index
directors.set_index('name', inplace=True)

Let's just concatenate the dataframe to itself for the sake of an example:

In [38]:
pd.concat([directors, directors], verify_integrity=True)

ValueError: Indexes have overlapping values: Index(['Jon Favreau', 'Joss Whedon', 'Anthony and Joe Russo',
       'Louis Leterrier', 'Kenneth Branagh', 'Joe Johnston', 'Shane Black',
       'Alan Taylor', 'James Gunn', 'Peyton Reed', 'Scott Derrickson',
       'Jon Watts'],
      dtype='object', name='name')

### Conclusion <small id='conclusion'></small>

Finally, the things I wanted to cover in this article are over. We talked about so many different methods for joining data. Believe it or not, there is still more to this topic such as `merge_ordered()` and `merge_asof()` methods but they might be a topic for another article. This just shows how deep and large `pandas` library is and what you can do with it. I intended this article as a one-stop place to learn about combining data. Hope you will use it in practice...